In [4]:
%pip install grakel pandas numpy matplotlib seaborn scikit-learn

  Using cached GraKeL-0.1.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import grakel as gk
import numpy as np
import pickle
from grakel import Graph
from grakel.utils import graph_from_networkx , cross_validate_Kfold_SVM
from grakel.kernels import WeisfeilerLehman, VertexHistogram, ShortestPath, ShortestPathAttr, Propagation, GraphHopper
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import train_test_split , cross_val_score
import seaborn as sns

#For Kaggle
data = pd.read_csv('/kaggle/input/ddosbalanced-with-new-attacks/DDosBalanced.csv')
dataNTP = pd.read_csv('/kaggle/input/ddosbalanced-with-new-attacks/BalancedNTP.csv')
dataSSDP = pd.read_csv('/kaggle/input/ddosbalanced-with-new-attacks/BalancedSSDP.csv')

#Outside Kaggle
# data = pd.read_csv('DDosBalanced.csv')
# dataNTP = pd.read_csv('BalancedNTP.csv')
# dataSSDP = pd.read_csv('BalancedSSDP.csv')

import warnings
warnings.filterwarnings('ignore') #to ignore the warning messages that come up when the kernel is used

#view the labels from the dataset
print(data[' Label'].value_counts())
print(dataNTP[' Label'].value_counts())
print(dataSSDP[' Label'].value_counts())


FileNotFoundError: [Errno 2] No such file or directory: 'DDosBalanced.csv'

<h1>For balanced Dataset, using test splits</h1>

In [3]:
graph_list = [] #Essentially storing the data
target_list = [] #To store the target of the data (the label) 1 being of Ddos_DNS, 0 being otherwise, 2 being Ddos_LDAP
testData = data
for i in range(len(data)): #Creates the graph of each entry in the data
    edges = {"Flow ID":["Source Port","Destination Port","Protocol","Flow Duration","Down/Up Ratio",
                         "URG Flag Count","Idle Mean","Active Mean","Initial Window Bytes Backward",
                         "Initial Window Bytes Forward","Flow Packets per Sec","Inbound"],
            "Flow Packets per Sec":["Flow ID","Backward IAT Total","Forward IAT Total",
                                    "Minimum Segment Size Forward","Average Packet Size"],
            "Source Port":["Flow ID","Total Forward Packets"],
            "Destination Port":["Flow ID","Total Backward Packets"],
            "Minimum Segment Size Forward":["Flow ID"],
            "Average Packet Size":["Flow ID"],
            "Total Forward Packets":["Source Port","Forward PSH Flags","Forward Header Length","Average Forward Segment Size","Forward Packets per Sec",
                                     "Total Length Forward Packets"],
            "Total Backward Packets":[ "Destination Port","Total Length Backward Packets","Backward Header Length",
                                      "Average Backward Segment Size","Backward Packets per Sec"],
            "Total Length Forward Packets":["Total Forward Packets"],
            "Total Length Backward Packets":["Total Backward Packets"],
            "Forward Header Length":["Total Forward Packets"],
            "Average Forward Segment Size":["Total Forward Packets"],
            "Forward Packets per Sec":["Total Forward Packets"],
            "Backward Header Length":["Total Backward Packets"],
            "Average Backward Segment Size":["Total Backward Packets"],
            "Backward Packets per Sec":["Total Backward Packets"],
            "Flow Duration":["Flow ID"],
            "Down/Up Ratio":["Flow ID"],
            "Active Mean":["Flow ID"],
            "Idle Mean":["Flow ID"],
            "Protocol":["Flow ID"],
            "Initial Window Bytes Backward":["Flow ID"],
            "Initial Window Bytes Forward":["Flow ID"],
            "Backward IAT Total":["Flow Packets per Sec"],
            "Forward IAT Total":["Flow Packets per Sec"],
            "Forward PSH Flags":["Total Forward Packets"],
            "URG Flag Count":["Flow ID"],
            "Inbound":["Flow ID"]}

    node_attributes = {
        #Look more into which data can be removed from correlation matrix
        "Flow ID":0, #testData["Flow ID"].values[i]
        "Destination Port":data[' Destination Port'].values[i],
        "Source Port":data[' Source Port'].values[i],
        "Minimum Segment Size Forward":data[' min_seg_size_forward'].values[i],
        "Average Packet Size":data[' Average Packet Size'].values[i],
        "Total Forward Packets":data[" Total Fwd Packets"].values[i],
        "Total Backward Packets":data[" Total Backward Packets"].values[i],
        "Total Length Forward Packets":data["Total Length of Fwd Packets"].values[i],
        "Total Length Backward Packets":data[" Total Length of Bwd Packets"].values[i],
        "Forward Header Length":data["Fwd Header Length"].values[i],
        "Average Forward Segment Size":data["Avg Fwd Segment Size"].values[i],
        "Forward Packets per Sec":data["Fwd Packets/s"].values[i],
        "Backward Header Length":data[" Bwd Header Length"].values[i],
        "Average Backward Segment Size":data["Avg Bwd Segment Size"].values[i],
        "Backward Packets per Sec":data[" Bwd Packets/s"].values[i],
        "Flow Duration":data[" Flow Duration"].values[i],
        "Down/Up Ratio":data["Down/Up Ratio"].values[i],
        "Active Mean":data["Active Mean"].values[i],
        "Idle Mean":data["Idle Mean"].values[i],
        "Initial Window Bytes Backward":data["Init_Win_bytes_backward"].values[i],
        "Initial Window Bytes Forward":data["Init_Win_bytes_forward"].values[i],
        "Flow Packets per Sec":data["Flow Packets/s"].values[i],
        "Backward IAT Total":data["Bwd IAT Total"].values[i],
        "Forward IAT Total":data["Fwd IAT Total"].values[i],
        "Protocol":data[" Protocol"].values[i],
        "Forward PSH Flags":data["Fwd PSH Flags"].values[i],
        "URG Flag Count":data["URG Flag Count"].values[i],
        "Inbound":data["Inbound"].values[i]
    }
    g = Graph(edges, node_labels = node_attributes)
    graph_list.append(g)
    target_list.append(data[" Label"].values[i]) #targets created

# For NTP    
for i in range(len(dataNTP)): #Creates the graph of each entry in the data
    edges = {"Flow ID":["Source Port","Destination Port","Protocol","Flow Duration","Down/Up Ratio",
                         "URG Flag Count","Idle Mean","Active Mean","Initial Window Bytes Backward",
                         "Initial Window Bytes Forward","Flow Packets per Sec","Inbound","ACK Flag Count", "SYN Flag Count", "CWE Flag Count"],
            "Flow Packets per Sec":["Flow ID","Backward IAT Total","Forward IAT Total",
                                    "Minimum Segment Size Forward","Average Packet Size"],
            "Source Port":["Flow ID","Total Forward Packets"],
            "Destination Port":["Flow ID","Total Backward Packets"],
            "Minimum Segment Size Forward":["Flow ID"],
            "Average Packet Size":["Flow ID"],
            "Total Forward Packets":["Source Port","Forward PSH Flags","Forward Header Length","Average Forward Segment Size","Forward Packets per Sec",
                                     "Total Length Forward Packets"],
            "Total Backward Packets":[ "Destination Port","Total Length Backward Packets","Backward Header Length",
                                      "Average Backward Segment Size","Backward Packets per Sec"],
            "Total Length Forward Packets":["Total Forward Packets"],
            "Total Length Backward Packets":["Total Backward Packets"],
            "Forward Header Length":["Total Forward Packets"],
            "Average Forward Segment Size":["Total Forward Packets"],
            "Forward Packets per Sec":["Total Forward Packets"],
            "Backward Header Length":["Total Backward Packets"],
            "Average Backward Segment Size":["Total Backward Packets"],
            "Backward Packets per Sec":["Total Backward Packets"],
            "Flow Duration":["Flow ID"],
            "Down/Up Ratio":["Flow ID"],
            "Active Mean":["Flow ID"],
            "Idle Mean":["Flow ID"],
            "Protocol":["Flow ID"],
            "Initial Window Bytes Backward":["Flow ID"],
            "Initial Window Bytes Forward":["Flow ID"],
            "Backward IAT Total":["Flow Packets per Sec"],
            "Forward IAT Total":["Flow Packets per Sec"],
            "Forward PSH Flags":["Total Forward Packets"],
            "URG Flag Count":["Flow ID"],
            "Inbound":["Flow ID"],
            "ACK Flag Count":["Flow ID"],
            "SYN Flag Count":["Flow ID"],
            "CWE Flag Count":["Flow ID"]}

    node_attributes = {
        #Look more into which data can be removed from correlation matrix
        "Flow ID":0, #testData["Flow ID"].values[i]
        "Destination Port":dataNTP[' Destination Port'].values[i],
        "Source Port":dataNTP[' Source Port'].values[i],
        "Minimum Segment Size Forward":dataNTP[' min_seg_size_forward'].values[i],
        "Average Packet Size":dataNTP[' Average Packet Size'].values[i],
        "Total Forward Packets":dataNTP[" Total Fwd Packets"].values[i],
        "Total Backward Packets":dataNTP[" Total Backward Packets"].values[i],
        "Total Length Forward Packets":dataNTP["Total Length of Fwd Packets"].values[i],
        "Total Length Backward Packets":dataNTP[" Total Length of Bwd Packets"].values[i],
        "Forward Header Length":dataNTP[" Fwd Header Length"].values[i],
        "Average Forward Segment Size":dataNTP[" Avg Fwd Segment Size"].values[i],
        "Forward Packets per Sec":dataNTP["Fwd Packets/s"].values[i],
        "Backward Header Length":dataNTP[" Bwd Header Length"].values[i],
        "Average Backward Segment Size":dataNTP[" Avg Bwd Segment Size"].values[i],
        "Backward Packets per Sec":dataNTP[" Bwd Packets/s"].values[i],
        "Flow Duration":dataNTP[" Flow Duration"].values[i],
        "Down/Up Ratio":dataNTP[" Down/Up Ratio"].values[i],
        "Active Mean":dataNTP["Active Mean"].values[i],
        "Idle Mean":dataNTP["Idle Mean"].values[i],
        "Initial Window Bytes Backward":dataNTP[" Init_Win_bytes_backward"].values[i],
        "Initial Window Bytes Forward":dataNTP["Init_Win_bytes_forward"].values[i],
        "Flow Packets per Sec":dataNTP[" Flow Packets/s"].values[i],
        "Backward IAT Total":dataNTP["Bwd IAT Total"].values[i],
        "Forward IAT Total":dataNTP["Fwd IAT Total"].values[i],
        "Protocol":dataNTP[" Protocol"].values[i],
        "Forward PSH Flags":dataNTP["Fwd PSH Flags"].values[i],
        "URG Flag Count":dataNTP[" URG Flag Count"].values[i],
        "Inbound":dataNTP[" Inbound"].values[i],
        "ACK Flag Count":dataNTP[" ACK Flag Count"].values[i],
        "SYN Flag Count":dataNTP[" SYN Flag Count"].values[i],
        "CWE Flag Count":dataNTP[" CWE Flag Count"].values[i]
    }
    g = Graph(edges, node_labels = node_attributes)
    graph_list.append(g)
    target_list.append(dataNTP[" Label"].values[i]) #targets created

#For SSDP graph      
for i in range(len(dataSSDP)): #Creates the graph of each entry in the data
    edges = {"Flow ID":["Source Port","Destination Port","Protocol","Flow Duration","Down/Up Ratio",
                         "URG Flag Count","Idle Mean","Active Mean","Initial Window Bytes Backward",
                         "Initial Window Bytes Forward","Flow Packets per Sec","Inbound","ACK Flag Count","CWE Flag Count"],
            "Flow Packets per Sec":["Flow ID","Backward IAT Total","Forward IAT Total",
                                    "Minimum Segment Size Forward","Average Packet Size"],
            "Source Port":["Flow ID","Total Forward Packets"],
            "Destination Port":["Flow ID","Total Backward Packets"],
            "Minimum Segment Size Forward":["Flow ID"],
            "Average Packet Size":["Flow ID"],
            "Total Forward Packets":["Source Port","Forward PSH Flags","Forward Header Length","Average Forward Segment Size","Forward Packets per Sec",
                                     "Total Length Forward Packets"],
            "Total Backward Packets":[ "Destination Port","Total Length Backward Packets","Backward Header Length",
                                      "Average Backward Segment Size","Backward Packets per Sec"],
            "Total Length Forward Packets":["Total Forward Packets"],
            "Total Length Backward Packets":["Total Backward Packets"],
            "Forward Header Length":["Total Forward Packets"],
            "Average Forward Segment Size":["Total Forward Packets"],
            "Forward Packets per Sec":["Total Forward Packets"],
            "Backward Header Length":["Total Backward Packets"],
            "Average Backward Segment Size":["Total Backward Packets"],
            "Backward Packets per Sec":["Total Backward Packets"],
            "Flow Duration":["Flow ID"],
            "Down/Up Ratio":["Flow ID"],
            "Active Mean":["Flow ID"],
            "Idle Mean":["Flow ID"],
            "Protocol":["Flow ID"],
            "Initial Window Bytes Backward":["Flow ID"],
            "Initial Window Bytes Forward":["Flow ID"],
            "Backward IAT Total":["Flow Packets per Sec"],
            "Forward IAT Total":["Flow Packets per Sec"],
            "Forward PSH Flags":["Total Forward Packets"],
            "URG Flag Count":["Flow ID"],
            "Inbound":["Flow ID"],
            "ACK Flag Count":["Flow ID"],
            "CWE Flag Count":["Flow ID"]}

    node_attributes = {
        #Look more into which data can be removed from correlation matrix
        "Flow ID":0, #testData["Flow ID"].values[i]
        "Destination Port":dataSSDP[' Destination Port'].values[i],
        "Source Port":dataSSDP[' Source Port'].values[i],
        "Minimum Segment Size Forward":dataSSDP[' min_seg_size_forward'].values[i],
        "Average Packet Size":dataSSDP[' Average Packet Size'].values[i],
        "Total Forward Packets":dataSSDP[" Total Fwd Packets"].values[i],
        "Total Backward Packets":dataSSDP[" Total Backward Packets"].values[i],
        "Total Length Forward Packets":dataSSDP["Total Length of Fwd Packets"].values[i],
        "Total Length Backward Packets":dataSSDP[" Total Length of Bwd Packets"].values[i],
        "Forward Header Length":dataSSDP[" Fwd Header Length"].values[i],
        "Average Forward Segment Size":dataSSDP[" Avg Fwd Segment Size"].values[i],
        "Forward Packets per Sec":dataSSDP["Fwd Packets/s"].values[i],
        "Backward Header Length":dataSSDP[" Bwd Header Length"].values[i],
        "Average Backward Segment Size":dataSSDP[" Avg Bwd Segment Size"].values[i],
        "Backward Packets per Sec":dataSSDP[" Bwd Packets/s"].values[i],
        "Flow Duration":dataSSDP[" Flow Duration"].values[i],
        "Down/Up Ratio":dataSSDP[" Down/Up Ratio"].values[i],
        "Active Mean":dataSSDP["Active Mean"].values[i],
        "Idle Mean":dataSSDP["Idle Mean"].values[i],
        "Initial Window Bytes Backward":dataSSDP[" Init_Win_bytes_backward"].values[i],
        "Initial Window Bytes Forward":dataSSDP["Init_Win_bytes_forward"].values[i],
        "Flow Packets per Sec":dataSSDP[" Flow Packets/s"].values[i],
        "Backward IAT Total":dataSSDP["Bwd IAT Total"].values[i],
        "Forward IAT Total":dataSSDP["Fwd IAT Total"].values[i],
        "Protocol":dataSSDP[" Protocol"].values[i],
        "Forward PSH Flags":dataSSDP["Fwd PSH Flags"].values[i],
        "URG Flag Count":dataSSDP[" URG Flag Count"].values[i],
        "Inbound":dataSSDP[" Inbound"].values[i],
        "ACK Flag Count":dataSSDP[" ACK Flag Count"].values[i],
        "CWE Flag Count":dataSSDP[" CWE Flag Count"].values[i]
    }
    g = Graph(edges, node_labels = node_attributes)
    graph_list.append(g)
    target_list.append(dataSSDP[" Label"].values[i]) #targets created
    
propagation_Kernel = Propagation(normalize = True)#[[1227.]], [[1219.]], [[1199.]]

G_train, G_test, t_train, t_test = train_test_split(graph_list, target_list, test_size=0.2) #look into cross validation

K_train = propagation_Kernel.fit_transform(G_train)
K_test = propagation_Kernel.transform(G_test)

94.00 %
[[208   0   0  26   5]
 [  1 244   1   6  20]
 [  0   0 293   0   2]
 [ 15   0   0 277   1]
 [  1   5   1   0 272]]


# Using OVO with Test Train splits

In [ ]:
clf = SVC() #is one vs one
clf.fit(K_train,t_train)
y_pred = clf.predict(K_test)
print("%2.2f %%" %(round(accuracy_score(t_test, y_pred)*100)))

conf_matrix = confusion_matrix(t_test,y_pred)

#From Seaborn documentation
    
plt.figure(figsize=(8,8))
sns.set(font_scale = 1.5)
 
ax = sns.heatmap( conf_matrix, annot=True, fmt='d', cbar=False,cmap='flag',vmax=150)
 
ax.set_xlabel("Predicted", labelpad=20)
ax.set_ylabel("Actual", labelpad=20)
plt.show()

# Using OVR with Test Train splits

In [ ]:
clf = LinearSVC(multi_class='ovr') #is one vs rest
clf.fit(K_train,t_train)
y_pred = clf.predict(K_test)
print("%2.2f %%" %(round(accuracy_score(t_test, y_pred)*100)))

conf_matrix = confusion_matrix(t_test,y_pred)

#From Seaborn documentation
    
plt.figure(figsize=(8,8))
sns.set(font_scale = 1.5)
 
ax = sns.heatmap( conf_matrix, annot=True, fmt='d', cbar=False,cmap='flag',vmax=150)
 
ax.set_xlabel("Predicted", labelpad=20)
ax.set_ylabel("Actual", labelpad=20)
plt.show()

<h1>For limited features, balanced Dataset, using cross validation</h1>

In [30]:
graph_list = [] #Essentially storing the data
target_list = [] #To store the target of the data (the label) 1 being of Ddos_DNS, 0 being otherwise, 2 being Ddos_LDAP
testData = data
for i in range(len(data)): #Creates the graph of each entry in the data
    edges = {"Flow ID":["Source Port","Destination Port","Protocol","Flow Duration","Down/Up Ratio",
                         "URG Flag Count","Idle Mean","Active Mean","Initial Window Bytes Backward",
                         "Initial Window Bytes Forward","Flow Packets per Sec","Inbound"],
            "Flow Packets per Sec":["Flow ID","Backward IAT Total","Forward IAT Total",
                                    "Minimum Segment Size Forward","Average Packet Size"],
            "Source Port":["Flow ID","Total Forward Packets"],
            "Destination Port":["Flow ID","Total Backward Packets"],
            "Minimum Segment Size Forward":["Flow ID"],
            "Average Packet Size":["Flow ID"],
            "Total Forward Packets":["Source Port","Forward PSH Flags","Forward Header Length","Average Forward Segment Size","Forward Packets per Sec",
                                     "Total Length Forward Packets"],
            "Total Backward Packets":[ "Destination Port","Total Length Backward Packets","Backward Header Length",
                                      "Average Backward Segment Size","Backward Packets per Sec"],
            "Total Length Forward Packets":["Total Forward Packets"],
            "Total Length Backward Packets":["Total Backward Packets"],
            "Forward Header Length":["Total Forward Packets"],
            "Average Forward Segment Size":["Total Forward Packets"],
            "Forward Packets per Sec":["Total Forward Packets"],
            "Backward Header Length":["Total Backward Packets"],
            "Average Backward Segment Size":["Total Backward Packets"],
            "Backward Packets per Sec":["Total Backward Packets"],
            "Flow Duration":["Flow ID"],
            "Down/Up Ratio":["Flow ID"],
            "Active Mean":["Flow ID"],
            "Idle Mean":["Flow ID"],
            "Protocol":["Flow ID"],
            "Initial Window Bytes Backward":["Flow ID"],
            "Initial Window Bytes Forward":["Flow ID"],
            "Backward IAT Total":["Flow Packets per Sec"],
            "Forward IAT Total":["Flow Packets per Sec"],
            "Forward PSH Flags":["Total Forward Packets"],
            "URG Flag Count":["Flow ID"],
            "Inbound":["Flow ID"]}

    node_attributes = {
        #Look more into which data can be removed from correlation matrix
        "Flow ID":0, #testData["Flow ID"].values[i]
        "Destination Port":data[' Destination Port'].values[i],
        "Source Port":data[' Source Port'].values[i],
        "Minimum Segment Size Forward":data[' min_seg_size_forward'].values[i],
        "Average Packet Size":data[' Average Packet Size'].values[i],
        "Total Forward Packets":data[" Total Fwd Packets"].values[i],
        "Total Backward Packets":data[" Total Backward Packets"].values[i],
        "Total Length Forward Packets":data["Total Length of Fwd Packets"].values[i],
        "Total Length Backward Packets":data[" Total Length of Bwd Packets"].values[i],
        "Forward Header Length":data["Fwd Header Length"].values[i],
        "Average Forward Segment Size":data["Avg Fwd Segment Size"].values[i],
        "Forward Packets per Sec":data["Fwd Packets/s"].values[i],
        "Backward Header Length":data[" Bwd Header Length"].values[i],
        "Average Backward Segment Size":data["Avg Bwd Segment Size"].values[i],
        "Backward Packets per Sec":data[" Bwd Packets/s"].values[i],
        "Flow Duration":data[" Flow Duration"].values[i],
        "Down/Up Ratio":data["Down/Up Ratio"].values[i],
        "Active Mean":data["Active Mean"].values[i],
        "Idle Mean":data["Idle Mean"].values[i],
        "Initial Window Bytes Backward":data["Init_Win_bytes_backward"].values[i],
        "Initial Window Bytes Forward":data["Init_Win_bytes_forward"].values[i],
        "Flow Packets per Sec":data["Flow Packets/s"].values[i],
        "Backward IAT Total":data["Bwd IAT Total"].values[i],
        "Forward IAT Total":data["Fwd IAT Total"].values[i],
        "Protocol":data[" Protocol"].values[i],
        "Forward PSH Flags":data["Fwd PSH Flags"].values[i],
        "URG Flag Count":data["URG Flag Count"].values[i],
        "Inbound":data["Inbound"].values[i]
    }
    g = Graph(edges, node_labels = node_attributes)
    graph_list.append(g)
    target_list.append([data[" Label"].values[i]]) #targets created

# For NTP    
for i in range(len(dataNTP)): #Creates the graph of each entry in the data
    edges = {"Flow ID":["Source Port","Destination Port","Protocol","Flow Duration","Down/Up Ratio",
                         "URG Flag Count","Idle Mean","Active Mean","Initial Window Bytes Backward",
                         "Initial Window Bytes Forward","Flow Packets per Sec","Inbound","ACK Flag Count", "SYN Flag Count", "CWE Flag Count"],
            "Flow Packets per Sec":["Flow ID","Backward IAT Total","Forward IAT Total",
                                    "Minimum Segment Size Forward","Average Packet Size"],
            "Source Port":["Flow ID","Total Forward Packets"],
            "Destination Port":["Flow ID","Total Backward Packets"],
            "Minimum Segment Size Forward":["Flow ID"],
            "Average Packet Size":["Flow ID"],
            "Total Forward Packets":["Source Port","Forward PSH Flags","Forward Header Length","Average Forward Segment Size","Forward Packets per Sec",
                                     "Total Length Forward Packets"],
            "Total Backward Packets":[ "Destination Port","Total Length Backward Packets","Backward Header Length",
                                      "Average Backward Segment Size","Backward Packets per Sec"],
            "Total Length Forward Packets":["Total Forward Packets"],
            "Total Length Backward Packets":["Total Backward Packets"],
            "Forward Header Length":["Total Forward Packets"],
            "Average Forward Segment Size":["Total Forward Packets"],
            "Forward Packets per Sec":["Total Forward Packets"],
            "Backward Header Length":["Total Backward Packets"],
            "Average Backward Segment Size":["Total Backward Packets"],
            "Backward Packets per Sec":["Total Backward Packets"],
            "Flow Duration":["Flow ID"],
            "Down/Up Ratio":["Flow ID"],
            "Active Mean":["Flow ID"],
            "Idle Mean":["Flow ID"],
            "Protocol":["Flow ID"],
            "Initial Window Bytes Backward":["Flow ID"],
            "Initial Window Bytes Forward":["Flow ID"],
            "Backward IAT Total":["Flow Packets per Sec"],
            "Forward IAT Total":["Flow Packets per Sec"],
            "Forward PSH Flags":["Total Forward Packets"],
            "URG Flag Count":["Flow ID"],
            "Inbound":["Flow ID"],
            "ACK Flag Count":["Flow ID"],
            "SYN Flag Count":["Flow ID"],
            "CWE Flag Count":["Flow ID"]}

    node_attributes = {
        #Look more into which data can be removed from correlation matrix
        "Flow ID":0, #testData["Flow ID"].values[i]
        "Destination Port":dataNTP[' Destination Port'].values[i],
        "Source Port":dataNTP[' Source Port'].values[i],
        "Minimum Segment Size Forward":dataNTP[' min_seg_size_forward'].values[i],
        "Average Packet Size":dataNTP[' Average Packet Size'].values[i],
        "Total Forward Packets":dataNTP[" Total Fwd Packets"].values[i],
        "Total Backward Packets":dataNTP[" Total Backward Packets"].values[i],
        "Total Length Forward Packets":dataNTP["Total Length of Fwd Packets"].values[i],
        "Total Length Backward Packets":dataNTP[" Total Length of Bwd Packets"].values[i],
        "Forward Header Length":dataNTP[" Fwd Header Length"].values[i],
        "Average Forward Segment Size":dataNTP[" Avg Fwd Segment Size"].values[i],
        "Forward Packets per Sec":dataNTP["Fwd Packets/s"].values[i],
        "Backward Header Length":dataNTP[" Bwd Header Length"].values[i],
        "Average Backward Segment Size":dataNTP[" Avg Bwd Segment Size"].values[i],
        "Backward Packets per Sec":dataNTP[" Bwd Packets/s"].values[i],
        "Flow Duration":dataNTP[" Flow Duration"].values[i],
        "Down/Up Ratio":dataNTP[" Down/Up Ratio"].values[i],
        "Active Mean":dataNTP["Active Mean"].values[i],
        "Idle Mean":dataNTP["Idle Mean"].values[i],
        "Initial Window Bytes Backward":dataNTP[" Init_Win_bytes_backward"].values[i],
        "Initial Window Bytes Forward":dataNTP["Init_Win_bytes_forward"].values[i],
        "Flow Packets per Sec":dataNTP[" Flow Packets/s"].values[i],
        "Backward IAT Total":dataNTP["Bwd IAT Total"].values[i],
        "Forward IAT Total":dataNTP["Fwd IAT Total"].values[i],
        "Protocol":dataNTP[" Protocol"].values[i],
        "Forward PSH Flags":dataNTP["Fwd PSH Flags"].values[i],
        "URG Flag Count":dataNTP[" URG Flag Count"].values[i],
        "Inbound":dataNTP[" Inbound"].values[i],
        "ACK Flag Count":dataNTP[" ACK Flag Count"].values[i],
        "SYN Flag Count":dataNTP[" SYN Flag Count"].values[i],
        "CWE Flag Count":dataNTP[" CWE Flag Count"].values[i]
    }
    g = Graph(edges, node_labels = node_attributes)
    graph_list.append(g)
    target_list.append([dataNTP[" Label"].values[i]]) #targets created

#For SSDP graph      
for i in range(len(dataSSDP)): #Creates the graph of each entry in the data
    edges = {"Flow ID":["Source Port","Destination Port","Protocol","Flow Duration","Down/Up Ratio",
                         "URG Flag Count","Idle Mean","Active Mean","Initial Window Bytes Backward",
                         "Initial Window Bytes Forward","Flow Packets per Sec","Inbound","ACK Flag Count","CWE Flag Count"],
            "Flow Packets per Sec":["Flow ID","Backward IAT Total","Forward IAT Total",
                                    "Minimum Segment Size Forward","Average Packet Size"],
            "Source Port":["Flow ID","Total Forward Packets"],
            "Destination Port":["Flow ID","Total Backward Packets"],
            "Minimum Segment Size Forward":["Flow ID"],
            "Average Packet Size":["Flow ID"],
            "Total Forward Packets":["Source Port","Forward PSH Flags","Forward Header Length","Average Forward Segment Size","Forward Packets per Sec",
                                     "Total Length Forward Packets"],
            "Total Backward Packets":[ "Destination Port","Total Length Backward Packets","Backward Header Length",
                                      "Average Backward Segment Size","Backward Packets per Sec"],
            "Total Length Forward Packets":["Total Forward Packets"],
            "Total Length Backward Packets":["Total Backward Packets"],
            "Forward Header Length":["Total Forward Packets"],
            "Average Forward Segment Size":["Total Forward Packets"],
            "Forward Packets per Sec":["Total Forward Packets"],
            "Backward Header Length":["Total Backward Packets"],
            "Average Backward Segment Size":["Total Backward Packets"],
            "Backward Packets per Sec":["Total Backward Packets"],
            "Flow Duration":["Flow ID"],
            "Down/Up Ratio":["Flow ID"],
            "Active Mean":["Flow ID"],
            "Idle Mean":["Flow ID"],
            "Protocol":["Flow ID"],
            "Initial Window Bytes Backward":["Flow ID"],
            "Initial Window Bytes Forward":["Flow ID"],
            "Backward IAT Total":["Flow Packets per Sec"],
            "Forward IAT Total":["Flow Packets per Sec"],
            "Forward PSH Flags":["Total Forward Packets"],
            "URG Flag Count":["Flow ID"],
            "Inbound":["Flow ID"],
            "ACK Flag Count":["Flow ID"],
            "CWE Flag Count":["Flow ID"]}

    node_attributes = {
        #Look more into which data can be removed from correlation matrix
        "Flow ID":0, #testData["Flow ID"].values[i]
        "Destination Port":dataSSDP[' Destination Port'].values[i],
        "Source Port":dataSSDP[' Source Port'].values[i],
        "Minimum Segment Size Forward":dataSSDP[' min_seg_size_forward'].values[i],
        "Average Packet Size":dataSSDP[' Average Packet Size'].values[i],
        "Total Forward Packets":dataSSDP[" Total Fwd Packets"].values[i],
        "Total Backward Packets":dataSSDP[" Total Backward Packets"].values[i],
        "Total Length Forward Packets":dataSSDP["Total Length of Fwd Packets"].values[i],
        "Total Length Backward Packets":dataSSDP[" Total Length of Bwd Packets"].values[i],
        "Forward Header Length":dataSSDP[" Fwd Header Length"].values[i],
        "Average Forward Segment Size":dataSSDP[" Avg Fwd Segment Size"].values[i],
        "Forward Packets per Sec":dataSSDP["Fwd Packets/s"].values[i],
        "Backward Header Length":dataSSDP[" Bwd Header Length"].values[i],
        "Average Backward Segment Size":dataSSDP[" Avg Bwd Segment Size"].values[i],
        "Backward Packets per Sec":dataSSDP[" Bwd Packets/s"].values[i],
        "Flow Duration":dataSSDP[" Flow Duration"].values[i],
        "Down/Up Ratio":dataSSDP[" Down/Up Ratio"].values[i],
        "Active Mean":dataSSDP["Active Mean"].values[i],
        "Idle Mean":dataSSDP["Idle Mean"].values[i],
        "Initial Window Bytes Backward":dataSSDP[" Init_Win_bytes_backward"].values[i],
        "Initial Window Bytes Forward":dataSSDP["Init_Win_bytes_forward"].values[i],
        "Flow Packets per Sec":dataSSDP[" Flow Packets/s"].values[i],
        "Backward IAT Total":dataSSDP["Bwd IAT Total"].values[i],
        "Forward IAT Total":dataSSDP["Fwd IAT Total"].values[i],
        "Protocol":dataSSDP[" Protocol"].values[i],
        "Forward PSH Flags":dataSSDP["Fwd PSH Flags"].values[i],
        "URG Flag Count":dataSSDP[" URG Flag Count"].values[i],
        "Inbound":dataSSDP[" Inbound"].values[i],
        "ACK Flag Count":dataSSDP[" ACK Flag Count"].values[i],
        "CWE Flag Count":dataSSDP[" CWE Flag Count"].values[i]
    }
    g = Graph(edges, node_labels = node_attributes)
    graph_list.append(g)
    target_list.append([dataSSDP[" Label"].values[i]]) #targets created
    
targetArray = np.array(target_list)

propagation_Kernel = Propagation(normalize = True)#[[1227.]], [[1219.]], [[1199.]]

kernelList = list()
gk = propagation_Kernel
kernelFit = gk.fit_transform(graph_list)
kernelList.append(kernelFit)

F1-Score 97.43%
Standard deviation: 0.0%


# To find the Accuracy from Cross Validation

In [6]:
accs = cross_validate_Kfold_SVM([kernelList], targetArray, n_iter=1, scoring='accuracy')
print("Accuracy", str(round(np.mean(accs[0])*100, 2)) + "%")

NameError: name 'kernelList' is not defined

# To find the F1-Score from Cross Validation

In [ ]:
accs = cross_validate_Kfold_SVM([kernelList], targetArray, n_iter=1, scoring='f1_macro')
print("F1-Score", str(round(np.mean(accs[0])*100, 2)) + "%")